In [5]:
import pandas as pd
df = pd.read_csv("../data_csv/페이스메이커.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   episode_id            15000 non-null  int64  
 1   step                  15000 non-null  int64  
 2   patient_id            15000 non-null  int64  
 3   age                   15000 non-null  int64  
 4   sex                   15000 non-null  int64  
 5   weight_kg             15000 non-null  float64
 6   diagnosis             15000 non-null  object 
 7   baseline_hr           15000 non-null  float64
 8   egfr                  15000 non-null  float64
 9   beta_blocker_dose_mg  15000 non-null  float64
 10  current_hr            15000 non-null  float64
 11  spo2                  15000 non-null  float64
 12  sbp                   15000 non-null  float64
 13  dbp                   15000 non-null  float64
 14  activity_level        15000 non-null  int64  
 15  arrhythmia_risk    

In [6]:
df.columns

Index(['episode_id', 'step', 'patient_id', 'age', 'sex', 'weight_kg',
       'diagnosis', 'baseline_hr', 'egfr', 'beta_blocker_dose_mg',
       'current_hr', 'spo2', 'sbp', 'dbp', 'activity_level', 'arrhythmia_risk',
       'prev_pacing_rate', 'action', 'new_pacing_rate', 'next_hr', 'next_sbp',
       'next_dbp', 'next_spo2', 'next_arrhythmia_risk', 'adverse_event',
       'reward'],
      dtype='object')

제어 전후 심박 안정화 확인 
* pacing 이후 심박이 baseline에 가까워지는가? 

In [7]:
df["hr_error_before"] = (df["current_hr"] - df["baseline_hr"]).abs()
df["hr_error_after"] = (df["next_hr"] - df["baseline_hr"]).abs()

df[["hr_error_before", "hr_error_after"]].describe()

,hr_error_before,hr_error_after
count,15000.000000,15000.000000
mean,19.587121,20.173581
std,13.347449,13.245013
min,0.000000,0.000061
25%,8.538912,9.402769
50%,17.539790,18.277367
75%,28.956238,29.404790
max,75.212533,75.212533


결과
* hr_error_after가 평균적으로 감소하지 않음 
* 오히려 소폭 증가 
* 항상 baseline에 수렴하도록 만들어진 데이터가 아니라
* 장기보상을 최대화하기 위해 설계됨 따라서 단기적으로 HR 오차가 늘어도 arrhythmia risk 감소나 adverse event 회피가 더 중요ㅗ하다면 그 방향으로 제어 
** 의사결정 trade-off가 존재함을 보여줌 

action(제어 여부)에 따른 효과 비교 -> EMS 개입 효과를 데이터로 보여줌 

In [8]:
df.groupby("action")[["hr_error_before", "hr_error_after", "reward"]].mean()

,hr_error_before,hr_error_after,reward
action,,,
-1,20.382359,20.254136,0.524136
0,19.985140,20.443194,0.638631
1,17.642252,19.213012,0.422173


* action = 0(유지)이 reward 평균이 가장 높음 
* action = +1, -1은 개입은 하지만 reward가 낮음 
* EMS/ 의료제어에서도 개입 자체가 항상 좋은 것은 아니다. 필요할 때만 개입하는 것이 최적 (스마트 그리드 EMS 논리와 일치 _ DR도 항상 쓰는 게 아니라 조건부로 씀)

arrhythmia_risk 구간별 제어 효과

In [9]:
df["risk_bin"] = pd.qcut(df["arrhythmia_risk"], q=3, labels=["low", "mid", "high"])

df.groupby("risk_bin")[["hr_error_before", "hr_error_after", "reward"]].mean()

/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_2790/3022972034.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("risk_bin")[["hr_error_before", "hr_error_after", "reward"]].mean()


,hr_error_before,hr_error_after,reward
risk_bin,,,
low,21.361045,21.419091,0.76912
mid,20.551478,20.920925,0.48352
high,16.848840,18.180727,0.48378


* high risk 구간 : 제어 후 HR error 증가, reward 중간 
* low risk 구간: reward 높음, HR error여도 상대적으로 큼 
* 고위험 환자에서는 HR을 baseline에 맞추는 것보다 리스크 억제가 우선되는 제어가 수행됨 

 ramp 개념을 심박에 적용 (전력 ↔ 의료 브릿지)

In [10]:
df["hr_ramp"] = (df["next_hr"] - df["current_hr"]).abs()

df.groupby("action")["hr_ramp"].mean()

action
-1    3.416390
 0    2.993404
 1    4.225808
Name: hr_ramp, dtype: float64

* action = 0 -> HR 변화가 가장 안정
* action = 1 -> 가장 큰 HR ramp
* 빠른 제어 = 항상 안정적인 건 아님 동적 안정성 (ramp 억제) 관점에서는 보수적 제어가 더 유리할 수도 있음 

reward 분포 비교

In [11]:
df.groupby("action")["reward"].describe()

,count,mean,std,min,25%,50%,75%,max
action,,,,,,,,
-1,2287.0,0.524136,0.626384,-2.0,-0.2,1.0,1.0,1.0
0,9777.0,0.638631,0.566832,-2.0,-0.2,1.0,1.0,1.0
1,2936.0,0.422173,0.646802,-2.0,-0.2,1.0,1.0,1.0


환자 특성별 분석도 가능

In [12]:
df.groupby("diagnosis")["reward"].mean().sort_values()

diagnosis
SICK_SINUS    0.538579
AV_BLOCK      0.541604
AF            0.640925
Name: reward, dtype: float64

본 의료 IoT 시뮬레이션 분석 결과,
제어 개입은 단일 안정성 지표(HR 오차)를 항상 개선하지는 않으며,
환자 상태(risk level)에 따라 서로 다른 목적 함수 간 trade-off가 존재함을 확인하였다.
이는 예측 기반 EMS가 단순한 안정화 장치가 아니라
상황 인지형 의사결정 시스템으로 설계되어야 함을 시사한다.